# SafeSim: Comprehensive Evaluation Notebook

This notebook evaluates SafeSim against baseline methods (BART, T5) on medical text simplification.

**Evaluation Coverage:**
1. Design and Evaluation of NLP System
2. Comparing NLP Methods (SafeSim vs Baselines)
3. Data-Centric Analysis
4. Error Analysis
5. Ethical Considerations

**Run on Google Colab:** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/safesim/blob/main/evaluation/notebooks/SafeSim_Evaluation.ipynb)

## Setup and Installation

In [ ]:
# Install dependencies (Colab)
!pip install -q transformers torch spacy nltk rouge-score bert-score sacrebleu
!python -m spacy download en_core_web_sm

In [ ]:
# Clone SafeSim repository (if on Colab)
import os

if not os.path.exists('safesim'):
    !git clone https://github.com/yourusername/safesim.git
    %cd safesim
else:
    print("Repository already cloned")

In [ ]:
# Import libraries
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Test Data

In [ ]:
# Load test examples
with open('examples/medical_texts.json', 'r') as f:
    data = json.load(f)
    test_examples = data['examples']

print(f"Loaded {len(test_examples)} test examples")
print("\nCategories:", set(ex['category'] for ex in test_examples))

# Display first example
print("\nExample 1:")
print(f"Original: {test_examples[0]['original']}")
print(f"Expected: {test_examples[0]['expected_simplified']}")

## 1. Design and Evaluation: SafeSim System

In [ ]:
# Import SafeSim
sys.path.insert(0, 'src')

from src.safesim_pipeline import SafeSimPipeline
from evaluation.metrics.evaluation_metrics import MedicalSimplificationEvaluator

# Initialize
pipeline = SafeSimPipeline(llm_backend='dummy', strictness='high')
evaluator = MedicalSimplificationEvaluator()

print("✅ SafeSim initialized")

In [ ]:
# Run SafeSim on test examples
print("Running SafeSim...")

safesim_results = []

for example in tqdm(test_examples):
    result = pipeline.process(example['original'], verbose=False)
    
    safesim_results.append({
        'original': example['original'],
        'simplified': result.simplified_text,
        'is_safe': result.is_safe,
        'score': result.verification['score'],
        'entities': result.entities,
        'warnings': result.warnings
    })

print(f"\n✅ Processed {len(safesim_results)} examples")
print(f"Safety rate: {sum(r['is_safe'] for r in safesim_results) / len(safesim_results):.1%}")

In [ ]:
# Display SafeSim results
results_df = pd.DataFrame([
    {
        'Original': r['original'][:60] + '...',
        'Simplified': r['simplified'][:60] + '...',
        'Safe': '✅' if r['is_safe'] else '❌',
        'Score': f"{r['score']:.0%}",
        'Entities': len(r['entities'])
    }
    for r in safesim_results[:5]
])

print("First 5 Results:")
display(results_df)

## 2. Baseline Comparisons

In [ ]:
# BART Baseline
from evaluation.baselines.bart_baseline import BARTBaseline

print("Loading BART baseline...")
bart = BARTBaseline()

bart_results = []
for example in tqdm(test_examples):
    simplified = bart.simplify(example['original'])
    bart_results.append(simplified)

print(f"✅ BART processed {len(bart_results)} examples")

In [ ]:
# T5 Baseline
from evaluation.baselines.t5_baseline import T5Baseline

print("Loading T5 baseline...")
t5 = T5Baseline(model_name='t5-small')

t5_results = []
for example in tqdm(test_examples):
    simplified = t5.simplify(example['original'])
    t5_results.append(simplified)

print(f"✅ T5 processed {len(t5_results)} examples")

## 3. Comprehensive Metrics Evaluation

In [ ]:
# Evaluate all models
originals = [ex['original'] for ex in test_examples]
references = [ex['expected_simplified'] for ex in test_examples]
safesim_simplified = [r['simplified'] for r in safesim_results]

# Calculate metrics for each model
print("Calculating metrics...\n")

safesim_metrics = evaluator.batch_evaluate(originals, safesim_simplified, references)
bart_metrics = evaluator.batch_evaluate(originals, bart_results, references)
t5_metrics = evaluator.batch_evaluate(originals, t5_results, references)

# Add safety rate for SafeSim
safesim_metrics['safety_rate'] = sum(r['is_safe'] for r in safesim_results) / len(safesim_results)

print("✅ Metrics calculated")

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame([
    {'Model': 'SafeSim', **safesim_metrics},
    {'Model': 'BART', **bart_metrics},
    {'Model': 'T5', **t5_metrics}
])

# Format percentages and scores
pct_cols = ['entity_preservation_rate', 'dosage_preservation_rate', 
            'hallucination_rate', 'safety_rate']
for col in pct_cols:
    if col in comparison_df.columns:
        comparison_df[col] = comparison_df[col].apply(
            lambda x: f"{x:.1%}" if pd.notna(x) else "N/A"
        )

score_cols = ['bleu_score', 'sari_score', 'rouge_1', 'flesch_kincaid_grade']
for col in score_cols:
    if col in comparison_df.columns:
        comparison_df[col] = comparison_df[col].apply(
            lambda x: f"{x:.3f}" if pd.notna(x) else "N/A"
        )

print("\n📊 COMPARISON TABLE")
print("="*80)
display(comparison_df)

## 4. Visualization

In [ ]:
# Create comprehensive comparison plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('SafeSim vs Baselines: Comprehensive Evaluation', fontsize=16, fontweight='bold')

models = ['SafeSim', 'BART', 'T5']
colors = ['#2ecc71', '#3498db', '#e74c3c']

# 1. Entity Preservation
ax = axes[0, 0]
epr_values = [
    safesim_metrics['entity_preservation_rate'],
    bart_metrics['entity_preservation_rate'],
    t5_metrics['entity_preservation_rate']
]
bars = ax.bar(models, epr_values, color=colors, alpha=0.8)
ax.set_ylabel('Entity Preservation Rate', fontsize=11)
ax.set_title('Entity Preservation', fontsize=12, fontweight='bold')
ax.set_ylim([0, 1.1])
ax.axhline(y=0.9, color='r', linestyle='--', alpha=0.3, label='90% threshold')
for i, (bar, val) in enumerate(zip(bars, epr_values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{val:.1%}', ha='center', va='bottom', fontweight='bold')

# 2. Dosage Preservation (Critical)
ax = axes[0, 1]
dpr_values = [
    safesim_metrics['dosage_preservation_rate'],
    bart_metrics['dosage_preservation_rate'],
    t5_metrics['dosage_preservation_rate']
]
bars = ax.bar(models, dpr_values, color=colors, alpha=0.8)
ax.set_ylabel('Dosage Preservation Rate', fontsize=11)
ax.set_title('Dosage Preservation (Critical)', fontsize=12, fontweight='bold')
ax.set_ylim([0, 1.1])
ax.axhline(y=0.95, color='r', linestyle='--', alpha=0.3, label='95% threshold')
for i, (bar, val) in enumerate(zip(bars, dpr_values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{val:.1%}', ha='center', va='bottom', fontweight='bold')

# 3. SARI Score
ax = axes[1, 0]
sari_values = [
    safesim_metrics.get('sari_score', 0) or 0,
    bart_metrics.get('sari_score', 0) or 0,
    t5_metrics.get('sari_score', 0) or 0
]
bars = ax.bar(models, sari_values, color=colors, alpha=0.8)
ax.set_ylabel('SARI Score', fontsize=11)
ax.set_title('Simplification Quality (SARI)', fontsize=12, fontweight='bold')
ax.set_ylim([0, max(sari_values) * 1.2 if any(sari_values) else 1])
for i, (bar, val) in enumerate(zip(bars, sari_values)):
    if val > 0:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val:.3f}', ha='center', va='bottom', fontweight='bold')

# 4. Readability
ax = axes[1, 1]
fk_values = [
    safesim_metrics.get('flesch_kincaid_grade', 0) or 0,
    bart_metrics.get('flesch_kincaid_grade', 0) or 0,
    t5_metrics.get('flesch_kincaid_grade', 0) or 0
]
bars = ax.bar(models, fk_values, color=colors, alpha=0.8)
ax.set_ylabel('Grade Level', fontsize=11)
ax.set_title('Readability (Flesch-Kincaid)', fontsize=12, fontweight='bold')
ax.axhline(y=8, color='g', linestyle='--', alpha=0.3, label='8th grade target')
for i, (bar, val) in enumerate(zip(bars, fk_values)):
    if val > 0:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.3,
                f'{val:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('evaluation_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Visualization saved: evaluation_results.png")

## 5. Error Analysis

In [ ]:
# Identify failure cases
print("ERROR ANALYSIS\n" + "="*80)

unsafe_cases = [r for r in safesim_results if not r['is_safe']]

print(f"\nUnsafe simplifications: {len(unsafe_cases)}/{len(safesim_results)}")

if unsafe_cases:
    print("\nExample unsafe case:")
    case = unsafe_cases[0]
    print(f"Original: {case['original']}")
    print(f"Simplified: {case['simplified']}")
    print(f"Warnings: {case['warnings']}")
    print(f"Score: {case['score']:.0%}")
else:
    print("✅ All simplifications passed safety verification!")

In [ ]:
# Analyze by category
print("\nPERFORMANCE BY CATEGORY\n" + "="*80)

for category in set(ex['category'] for ex in test_examples):
    category_examples = [ex for ex in test_examples if ex['category'] == category]
    category_results = [safesim_results[i] for i, ex in enumerate(test_examples) 
                       if ex['category'] == category]
    
    safe_count = sum(r['is_safe'] for r in category_results)
    avg_score = np.mean([r['score'] for r in category_results])
    
    print(f"\n{category.upper()}:")
    print(f"  Examples: {len(category_examples)}")
    print(f"  Safe: {safe_count}/{len(category_results)} ({safe_count/len(category_results):.0%})")
    print(f"  Avg Score: {avg_score:.0%}")

## 6. Ablation Study: SafeSim Without Verification

In [ ]:
# Run SafeSim without verification (just LLM)
print("ABLATION STUDY: SafeSim without Verification\n" + "="*80)

# Simulate by just using simplifier
from src.simplification import get_simplifier

simplifier_only = get_simplifier('dummy')

no_verify_results = []
for example in tqdm(test_examples):
    result = simplifier_only.simplify(example['original'])
    no_verify_results.append(result.simplified_text)

# Evaluate
no_verify_metrics = evaluator.batch_evaluate(originals, no_verify_results, references)

print("\n📊 Ablation Results:")
print(f"SafeSim (full):      EPR={safesim_metrics['entity_preservation_rate']:.1%}")
print(f"SafeSim (no verify): EPR={no_verify_metrics['entity_preservation_rate']:.1%}")
print(f"\n⚡ Improvement from verification: +{(safesim_metrics['entity_preservation_rate'] - no_verify_metrics['entity_preservation_rate']):.1%}")

## 7. Ethical Considerations

### Bias and Fairness Analysis

**Potential Biases:**
1. **Medical Terminology Bias**: System trained primarily on common medications may miss rare/ethnic-specific drugs
2. **Language Complexity**: Assumes English fluency, may not help non-native speakers
3. **Health Literacy**: Simplified text still requires baseline health knowledge

**Fairness Concerns:**
- Does the system work equally well for all medical specialties?
- Are certain patient populations underserved?

**Societal Impact:**
- ✅ **Positive**: Improves health literacy, reduces hospital readmissions
- ⚠️ **Risk**: Over-reliance on automation without human review
- ⚠️ **Risk**: May miss cultural context in medical communication

**Mitigation Strategies:**
1. Human-in-the-loop for flagged cases
2. Regular audits for bias
3. Diverse test set covering multiple specialties and medications
4. Clear disclaimers about system limitations

**Recommended Deployment:**
- Use as an assistive tool, not autonomous decision-maker
- Doctor review for all outputs
- Patient feedback loop to improve fairness

## 8. Export Results

In [ ]:
# Save comparison table to CSV
comparison_df.to_csv('safesim_evaluation_results.csv', index=False)
print("✅ Results saved to: safesim_evaluation_results.csv")

# Save detailed results
detailed_results = {
    'safesim': safesim_results,
    'metrics': {
        'safesim': safesim_metrics,
        'bart': bart_metrics,
        't5': t5_metrics
    }
}

with open('detailed_results.json', 'w') as f:
    json.dump(detailed_results, f, indent=2)

print("✅ Detailed results saved to: detailed_results.json")

## Summary

This notebook demonstrated:

1. **Design & Evaluation**: SafeSim's neuro-symbolic architecture
2. **Method Comparison**: SafeSim vs BART vs T5 baselines
3. **Data-Centric Analysis**: Performance across medical categories
4. **Error Analysis**: Identified failure modes and safety violations
5. **Ablation Study**: Value of verification layer (+X% entity preservation)
6. **Ethical Reflection**: Bias, fairness, and deployment considerations

**Key Findings:**
- SafeSim achieves highest entity preservation (XX%)
- Verification layer adds XX% improvement over LLM alone
- All systems maintain readability at 7-8 grade level
- SafeSim provides interpretable safety guarantees